# An Introduction to Linear Learner with MNIST
## Making a Binary Prediction of Whether a Handwritten Digit is a 0

This an example to use Amazon Sagemaker. SageMaker allows one to build a ML pipeline easily. Building, training and deploying of ML models is less cumbersome with SageMaker

In this example, I will be using a Amazon's marketplace algorithm (Linear Learner).
The purpose of the notebook is to explain the usage of sagemaker and not the modeling aspect.The data used in this is mnist and problem is framed as binary classification.


Amazon SageMaker's Linear Learner algorithm extends upon typical linear models by training many models in parallel, in a computationally efficient manner. Each model has a different set of hyperparameters, and then the algorithm finds the set that optimizes a specific criteria. This can provide substantially more accurate models than typical linear algorithms at the same, or lower, cost.

## Libraries used

In [3]:
import boto3
import re
import pickle 
import gzip
import numpy as np 
import urllib.request
import json
from sagemaker import get_execution_role


This notebook assumes that you have an AWS account and an IAM user setup and using the notebook instance of Amazon SageMaker. For further reference please refer to this notebook https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks.html

## Permissions and environment variables


This notebook was created and tested on an ml.t2.medium notebook instance.

Let's start by specifying:

1. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
2. The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. 

In [4]:
bucket = 'test-karan-02'
prefix = 'sagemaker_demo'
role = get_execution_role()